## JITEN MISHRA DSC430_Assignment0701_WarAndPeace
### I have not given or received any unauthorized assistance on this assignment.

#### Logic of Pseudo random number generation

* In this logic of random number generation we are reading a famous novel and comparing characters to generate  a random number based on 32 bit (custom logic) encryption.
* In order to do so we are maintain few variables to traverse the file smoothly in initial and subsequent calls.
* like the data for seed, step, startpoint, end point
* These variables are initialized along with other when a class object is created.
* The random funciton in the class calls the getBitList to get the list of 1's and 0's
* the call condition is based on two scenario
    * When the call is a 1st call of the object having start position as 0
    * when the call is a subsequent call of the object where the start poistion will be the last read position
    * If the provided seed value is greater than the end of file then the start position is wrapped to the begining of the file.
* Accourding to the above condition the getBitList is called with proper input variables.
* The getBitList funcition reads the file and traverse through it to get the charactes and compare them and store 1's or 0's to a list. We ignore the condition when the characters are same.
    * The reading logic is based on few condition in order to honor if the run is a 1st run or repeated run for 32 bit generation or if the function is called repeatedly for the same class object.
    * It seeks the position to the required spot
    * While being in a loop to gather 32 bits it checks the current postion and next position based on condition whether its a 1st run or a subsequent run it gets the characters.
    * The logic also handles the case when the end of file is reached in a run based on seed value and step value
    * It compares the next position with the end postion and if its beyond the end position it wraps the pointer to exact same number of position that it exceeds from the end.
    * It than reads the Char and compares them if the 1st char > 2nd char then 1 is appended to the list else 0.
* This received list from the getBitList method is then passed to the calcRandNum to compute the random number.
    * Calculation is as such. 
    * if the list is [1,0,1,1,0.....]
    * then calculation is 1*1/2 + 0*1/4 + 1*1/8 + 1*1/16 + 0*1/32 ......
* Finally the random number generated is returned back.

**Note : The class name is changed to shorten it as suggested by Professor**

In [3]:
# import statements
import random
from statistics import mean
import numpy as np

In [8]:
class WarAndPeacePrng:
    """
    Class to generate a Pseudo Random Number 
    with 32 bit encryption 
    """
    
    def __init__(self, seed=0, step=100):
        """
        Initialize the seed, step, bits, start point
        and the end position of the file
        seed : defualted to 0 , hold seed value to read from
        step : default to 100an offset value to skip from 
               current position in file
        """
        
        # initialize the variables
        self.seed = seed
        self.step = step
        self.fileName= 'war-and-peace.txt'
        self.bits = 32
        self.startPoint = -1
        
        # open the file
        # get the end position of the file
        # and move to fist position back
        self.file = open(self.fileName,'rb')
        self.file.seek(0,2) 
        self.endPoint = self.file.tell()
        self.file.seek(0,0)
        
    def random(self):
        """
        Function to get the bist list based on the algorithm
        And generate a random number
        """
        
        # if this is a first run then startpoint is -1
        # so the getBitList is called by passing actual seed value
        # else the seed value is passes as 0 because this will be 
        # a subsequent call of random function in the same object.
        if(self.startPoint == -1):
            # check the validity of the step and start point
            # compute it for valid traverse of file
            self.step,self.seed = WarAndPeacePrng.checkPosValidity(self.seed, self.step,
                                                                   self.endPoint)
            
            # in case of initial run it will start from seed position
            # in case of subsequent call it will start from last left position
            self.file.seek(self.seed,0)
        else:
            self.seed = 0
            
        bitList,startPos = WarAndPeacePrng.getBitList(self.bits,
                                                      self.step, self.startPoint,
                                                      self.endPoint, self.file)
        
        # set the start point received to the class variable.
        self.startPoint = startPos
        
        # traverse the bitList and calculate the randome number
        rndNum = WarAndPeacePrng.calcRandNum(bitList)
        
        # return the random number calculated
        return rndNum
    
    def close(self):
        """
        Function to close the opened file
        """
        
        if self.file:
            self.file.close()
            self.file = None
        
        
    def checkPosValidity(seed,step,endPoint):
        """
        Function to compute the validity of startPoint and step 
        seed       : seed value for traversing
        step       : step value for traversing
        startPoint : starat position for file traversal
        endPoint   : end point of the file.
        """
        
        # if the seed value is more than the end of file
        # set the start point to the wraped position
        if(seed >= endPoint):
            seed = seed % endPoint
            
        # if the step provided is greater than the end of file
        # set the step to wrapped position from the begining of file
        if(step >= endPoint):
            step = step % endPoint
        
        return step,seed
            
        
        
    def getBitList(bits, step, startPos, endPos, file):
        """
        Function to get the list of 1's and 0's
        fileName : Name of the file
        bits     : bit of encryption , here it is 32
        seed     : seed to read the file from position in initial run.
        startPos : position of file in the run 
        endPos   : the last position of the file
        """
                
        # Inittialize  a list to hold the 1's and 0's
        lsBit = []
                            
        loop = 1
        
        # loop for 32 iterations as per the bit definition
        # get two character from file and compute the 1's and 0's
        while loop <= bits:
                          
            # read the 1st Char :
            # if this is a first run
            # get the character  and proceed
            # else compute the place with end of file
            # move the pointer as per step / wrap position 
            # and then read the character
            # everytime we read a char the pointer moves so we take it 1 position back 
            if(startPos != -1):
                file = WarAndPeacePrng.computePlace(file, endPos, step)
            charA = file.read(1)
            file.seek(-1,1)
            
            # read the 2nd Char :
            # check the next position to read 
            # check the end of file for next character to read
            # move the pointer as per step from current position
            # and read the char
            # everytime we read a char the pointer moves so we take it 1 position back 
            file = WarAndPeacePrng.computePlace(file, endPos, step)
            charB = file.read(1)
            file.seek(-1,1)
            
            # compare the character's and append the list
            # no append when the characters are same
            if charA > charB :
                lsBit.append(1)
            elif charA != charB:
                lsBit.append(0)
            
            # if the characters are not same 
            # only then increase the loop counter
            # in oreder to get 32 bits
            if charA != charB:
                loop += 1
            
            # set the start point to the last pointer position
            # used for the next run
            startPos = file.tell()
        
        # return the list and the start position
        return lsBit,startPos
    
    def computePlace(file, endPos, step):
        """
        Funciton to move to the next position to read the character
        """
            
        # if the next position is greater than end of file
        # wrap up the position if true
        # else seek to next position
        nxtPos = file.tell() + step
        if(nxtPos >= endPos):
            nxtPos = nxtPos - endPos
            file.seek(nxtPos,0)
        else:
            file.seek(step,1)
            
        return file
        
    def calcRandNum(lsBit):
        """
        Function to calculated the random nuber
        lsBit : list of bits 
        """
        
        # initialize bit rate as 2
        bitRate = 2
        
        value = 0
        
        # loop the list and calculate the value 
        # as per bit rate
        # ex: 1*1/2 + 0*1/4 + 1*1/8 ...
        for ls in lsBit:
            value += ls*(1/bitRate)
            bitRate +=bitRate
        
        # return the value
        return value
                   

In [21]:
# test with default Constructor
wp = WarAndPeacePrng()  
print(wp.random())
wp.close()

0.9871646624524146


* One time random number generated for defualt constructor ( default : seed 0 , step = 100 )
0.9871646624524146

####################################################################################################################

In [4]:
# test with seed value
wp = WarAndPeacePrng(1000)  
print(wp.random())
wp.close()

0.5892692035995424


* One time random number generated for parameterized constructor ( seed value = 1000 )
0.5892692035995424

####################################################################################################################

In [9]:
# test 1 : seed = 0 and step = 100
wp = WarAndPeacePrng()

lsRan = []
for loop in range(10000):
    lsRan.append(wp.random())

wp.close()

print("The minimum of the all the random numbers is::: "+str(min(lsRan)))
print("The maximum of the all the random numbers is::: "+str(max(lsRan)))
print("The mean of the all the random numbers is::: "+str(mean(lsRan)))


The minimum of the all the random numbers is::: 0.000310925068333745
The maximum of the all the random numbers is::: 0.9997792935464531
The mean of the all the random numbers is::: 0.5000696333282162


 * The range of the random number is distributed between 0 upto 1 [0,1)
 * with an average of 0.5
 * The random numbers are spead accross with a min = 0.000310925068333745 , max = 0.9997792935464531 with and average of 0.5000696333282162

####################################################################################################################

In [6]:
# test 2 : seed = 1000 and step = 1000
wp = WarAndPeacePrng(1000,1000)

lsRan = []
for loop in range(10000):
    lsRan.append(wp.random())

wp.close()

print("The minimum of the all the random numbers is::: "+str(min(lsRan)))
print("The maximum of the all the random numbers is::: "+str(max(lsRan)))
print("The mean of the all the random numbers is::: "+str(mean(lsRan)))


The minimum of the all the random numbers is::: 7.051276043057442e-06
The maximum of the all the random numbers is::: 0.9999653820414096
The mean of the all the random numbers is::: 0.5007472428745823


**Please Note the min has e-05**
* The range of the random number is distributed between 0 upto 1 [0,1)
* with an average of 0.5
* The random numbers are spead accross with a min = 7.051276043057442e-06 , max = 0.9999653820414096 with and average of 0.5007472428745823

####################################################################################################################

In [14]:
# test 3 : seed = 2000 and step = 10000
wp = WarAndPeacePrng(2000,10000)

lsRan = []
for loop in range(10000):
    lsRan.append(wp.random())

wp.close()

print("The minimum of the all the random numbers is::: "+str(float(min(lsRan))))
print("The maximum of the all the random numbers is::: "+str(max(lsRan)))
print("The mean of the all the random numbers is::: "+str(mean(lsRan)))


The minimum of the all the random numbers is::: 2.1524261683225632e-05
The maximum of the all the random numbers is::: 0.9998614604119211
The mean of the all the random numbers is::: 0.5014544214995811


**Please Note the min has e-05** 
* The range of the random number is distributed between 0 upto 1 [0,1)
* with an average of 0.5
* The random numbers are spead accross with a min = 2.1524261683225632e-05 , max = 0.9998614604119211 with and average of 0.5025371074110735

####################################################################################################################

In [16]:
# test 4 : Average of 4 runs of seed = 2000 and step = 1000

wp = WarAndPeacePrng(2000,1000)

lsRan1 = []
lsRan2 = []
lsRan3 = []
lsRan4 = []

for loop in range(10000):
    lsRan1.append(wp.random())

for loop in range(10000):
    lsRan2.append(wp.random())

for loop in range(10000):
    lsRan3.append(wp.random())

for loop in range(10000):
    lsRan4.append(wp.random())
    
wp.close()
    
min1 = min(lsRan1)
max1 = max(lsRan1)
mean1 = mean(lsRan1)
    
print("The minimum of the random numbers of run 1 ::: "+str(min1))
print("The maximun of the random numbers of run 1::: "+str(max1))
print("The mean/average of the random numbers of run 1::: "+str(mean1))
print("****************************************************************")

min2 = min(lsRan2)
max2 = max(lsRan2)
mean2 = mean(lsRan2)

print("The minimum of the random numbers of run 2 ::: "+str(min2))
print("The maximun of the random numbers of run 2::: "+str(max2))
print("The mean/average of the random numbers of run 2::: "+str(mean2))
print("****************************************************************")

min3 = min(lsRan3)
max3 = max(lsRan3)
mean3 = mean(lsRan3)

print("The minimum of the random numbers of run 3 ::: "+str(min3))
print("The maximun of the random numbers of run 3::: "+str(max3))
print("The mean/average of the random numbers of run 3::: "+str(mean3))
print("****************************************************************")

min4 = min(lsRan4)
max4 = max(lsRan4)
mean4 = mean(lsRan4)

print("The minimum of the random numbers of run 4 ::: "+str(min4))
print("The maximun of the random numbers of run 4::: "+str(max4))
print("The mean/average of the random numbers of run 4::: "+str(mean4))
print("****************************************************************")


print("\nAverage of the mins , maxs and means of the 4 runs")

avgMin = mean([min1,min2,min3,min4])
avgMax = mean([max1,max2,max3,max4])
avgMean = mean([mean1,mean2,mean3,mean4])

print("Average Minimun of 4 runs:::"+str(avgMin))
print("Average Maximun of 4 runs:::"+str(avgMax))
print("Average mean of 4 runs:::"+str(avgMean))


The minimum of the random numbers of run 1 ::: 8.551636710762978e-06
The maximun of the random numbers of run 1::: 0.9997559133917093
The mean/average of the random numbers of run 1::: 0.5023259551132331
****************************************************************
The minimum of the random numbers of run 2 ::: 2.6056542992591858e-05
The maximun of the random numbers of run 2::: 0.9999250811524689
The mean/average of the random numbers of run 2::: 0.4998037668521982
****************************************************************
The minimum of the random numbers of run 3 ::: 8.709728717803955e-06
The maximun of the random numbers of run 3::: 0.9999958125408739
The mean/average of the random numbers of run 3::: 0.5013523014900274
****************************************************************
The minimum of the random numbers of run 4 ::: 0.0002590916119515896
The maximun of the random numbers of run 4::: 0.9998360644094646
The mean/average of the random numbers of run 4::: 0.50105

**Please not the min's has e-05**

* The range of the random number are distributed from 0 upto 1 [0,1)
* With an average of 0.5

* The above two statement stands true for each individual runs
* Also the average of the 4 runs mins = 7.56023800931871e-05 , maxs = 0.9998782178736292 and means = 0.5011345315031125 which stands true to the above statements

In [15]:
# Extra test  : 4 runs of seed = 2000 and step = 1000

wp = WarAndPeacePrng(2000,1000)

lsRan1 = []

for loop in range(10000):
    lsRan1.append(wp.random())

for loop in range(10000):
    lsRan1.append(wp.random())

for loop in range(10000):
    lsRan1.append(wp.random())

for loop in range(10000):
    lsRan1.append(wp.random())

wp.close()
    
print("The minimum of 4 runs of the random numbers is::: "+str(min(lsRan1)))
print("The maximun of 4 runs of the random numbers is::: "+str(max(lsRan1)))
print("The mean/average of 4 runs of the random numbers is::: "+str(mean(lsRan1)))

The minimum of 4 runs of the random numbers is::: 8.551636710762978e-06
The maximun of 4 runs of the random numbers is::: 0.9999958125408739
The mean/average of 4 runs of the random numbers is::: 0.5011345315031125


**Please Note the min has e-05**

* The range of the random number is distributed between 0 upto 1 [0,1)
* with an average of 0.5
* The random numbers are spead accross with a min = 8.551636710762978e-06 , max = 0.9999958125408739 with and average of 0.5011345315031125

####################################################################################################################